# 1. Introduction

In [7]:
import pandas as pd

world_dev = pd.read_csv("World_dev.csv")
col_renaming = {'SourceOfMostRecentIncomeAndExpenditureData': 'IESurvey'}

happiness_2015=pd.read_csv("World_Happiness_2015.csv")

merged=pd.merge(happiness_2015,world_dev, how="left", left_on="Country", right_on="ShortName")


merged.rename(col_renaming,axis=1,inplace=True)

print(merged)

         Country                         Region_x  Happiness Rank  \
0    Switzerland                   Western Europe               1   
1        Iceland                   Western Europe               2   
2        Denmark                   Western Europe               3   
3         Norway                   Western Europe               4   
4         Canada                    North America               5   
..           ...                              ...             ...   
153       Rwanda               Sub-Saharan Africa             154   
154        Benin               Sub-Saharan Africa             155   
155        Syria  Middle East and Northern Africa             156   
156      Burundi               Sub-Saharan Africa             157   
157         Togo               Sub-Saharan Africa             158   

     Happiness Score  Standard Error  Economy (GDP per Capita)   Family  \
0              7.587         0.03411                   1.39651  1.34951   
1              7.561 

# 2. Using Apply to Transform Strings

In [9]:
def extract_last_word(element):
    return str(element).split()[-1]

merged["Currency Apply"]=merged["CurrencyUnit"].apply(extract_last_word)

print(merged.head())

       Country        Region_x  Happiness Rank  Happiness Score  \
0  Switzerland  Western Europe               1            7.587   
1      Iceland  Western Europe               2            7.561   
2      Denmark  Western Europe               3            7.527   
3       Norway  Western Europe               4            7.522   
4       Canada   North America               5            7.427   

   Standard Error  Economy (GDP per Capita)   Family  \
0         0.03411                   1.39651  1.34951   
1         0.04884                   1.30232  1.40223   
2         0.03328                   1.32548  1.36058   
3         0.03880                   1.45900  1.33095   
4         0.03553                   1.32629  1.32261   

   Health (Life Expectancy)  Freedom  Trust (Government Corruption)  ...  \
0                   0.94143  0.66557                        0.41978  ...   
1                   0.94784  0.62877                        0.14145  ...   
2                   0.87464  0.6

# 3. Vectorized String Methods Overview

Whenever possible, we should try to use the vectorized methods instead of Series.apply() due to performance reasons.

Here are some examples.

| Method               | Description                                                   |
|----------------------|---------------------------------------------------------------|
| Series.str.split()   | Splits each element in the Series.                            |
| Series.str.strip()   | Strips whitespace from each string in the Series.             |
| Series.str.lower()   | Converts strings in the Series to lowercase.                  |
| Series.str.upper()   | Converts strings in the Series to uppercase.                  |
| Series.str.get()     | Retrieves the ith element of each element in the Series.      |
| Series.str.replace() | Replaces a regex or string in the Series with another string. |
| Series.str.cat()     | Concatenates strings in a Series.                             |
| Series.str.extract() | Extracts substrings from the Series matching a regex pattern. |



In [12]:
merged["Currency Vectorized"]=merged["CurrencyUnit"].str.split().str.get(-1)

print(merged["Currency Vectorized"].head())

0     franc
1     krona
2     krone
3     krone
4    dollar
Name: Currency Vectorized, dtype: object


# 4. Exploring Missing Values with Vectorized String Methods

In [17]:
print(merged["CurrencyUnit"].isnull().sum())

lengths=merged["CurrencyUnit"].str.len()

value_counts=lengths.value_counts(dropna=False)

print(value_counts)

13
14.0    21
4.0     20
12.0    17
13.0    14
NaN     13
15.0    13
16.0    12
18.0     9
17.0     9
11.0     8
22.0     7
25.0     5
19.0     3
9.0      2
10.0     1
39.0     1
26.0     1
23.0     1
20.0     1
Name: CurrencyUnit, dtype: int64


# 5. Finding Specific Words in Strings

In [19]:
pattern = r"[Nn]ational accounts"

national_accounts=merged["SpecialNotes"].str.contains(pattern)

print(national_accounts.head())

0     NaN
1     NaN
2     NaN
3     NaN
4    True
Name: SpecialNotes, dtype: object


# 6. Finding Specific Words in Strings Continue

In [21]:
pattern = r"[Nn]ational accounts"

national_accounts=merged["SpecialNotes"].str.contains(pattern,na=False)

merged_national_accounts=merged[national_accounts]

print(merged_national_accounts.head())

          Country                   Region_x  Happiness Rank  Happiness Score  \
4          Canada              North America               5            7.427   
7          Sweden             Western Europe               8            7.364   
8     New Zealand  Australia and New Zealand               9            7.286   
9       Australia  Australia and New Zealand              10            7.284   
14  United States              North America              15            7.119   

    Standard Error  Economy (GDP per Capita)   Family  \
4          0.03553                   1.32629  1.32261   
7          0.03157                   1.33171  1.28907   
8          0.03371                   1.25018  1.31967   
9          0.04083                   1.33358  1.30923   
14         0.03839                   1.39451  1.24711   

    Health (Life Expectancy)  Freedom  Trust (Government Corruption)  ...  \
4                    0.90563  0.63297                        0.32957  ...   
7               

# 7. Extracting Substrings from a Series

In [24]:
pattern =r"([0-2][0-9]{3})"

years=merged["SpecialNotes"].str.extract(pattern)

print(years)

        0
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
..    ...
153  2006
154   NaN
155   NaN
156   NaN
157  2013

[158 rows x 1 columns]


# 8. Extracting Substrings from a Series Continued

In the last exercise, we learned how to identify more complex patterns with regular expressions and extract substrings from a column using that pattern.

When we used the Series.str.extract() method, we enclosed our regular expression in parentheses. The parentheses indicate that only the character pattern matched should be extracted and returned in a series. We call this a capturing group.

If the capturing group doesn't exist in a row (or there is no match) the value in that row is set to NaN instead. As a result, the Series returned looked like this:



# 9. Extracting All Matches of a Pattern from a Series

In [27]:
pattern = r"(?P<Years>[1-2][0-9]{3})"

years=merged["IESurvey"].str.extractall(pattern)

value_counts=years["Years"].value_counts()

print(value_counts)

2012    33
2010    28
2011    22
2013    12
2009     8
2005     6
2008     6
2007     4
2004     3
2002     1
1992     1
2006     1
2003     1
1998     1
1995     1
2000     1
1999     1
Name: Years, dtype: int64


# 10. Extracting More Than One Group of Patterns from a Series

In [ ]:
pattern = r"(?P<First_Year>[1-2][0-9]{3})/?(?P<Second_Year>[0-9]{2})?"

